In [ ]:
# Copyright (c) 2012-2023, NECOTIS
# All rights reserved.
#
# Redistribution and use in source and binary forms, with or without modification,
# are permitted provided that the following conditions are met:
#
#  - Redistributions of source code must retain the above copyright notice,
#    this list of conditions and the following disclaimer.
#  - Redistributions in binary form must reproduce the above copyright notice,
#    this list of conditions and the following disclaimer in the documentation
#    and/or other materials provided with the distribution.
#  - Neither the name of the copyright holder nor the names of its contributors
#    may be used to endorse or promote products derived from this software
#    without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND
# ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED
# WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED.
# IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT,
# INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT
# NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA,
# OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY,
# WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE)
# ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE
# POSSIBILITY OF SUCH DAMAGE.

# Authors: Emmanuel Calvet, Jean Rouat, Bertrand Reulet (advisor)
# Date: July 07th, 2023
# Organization: Groupe de recherche en Neurosciences Computationnelles et Traitement Intelligent des Signaux (NECOTIS),
# Université de Sherbrooke, Canada


In [ ]:
# Import library and models
# NB: you need to import binary_model into the python path
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Standard library
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# The model
from tasks.whitenoise import whiteNoiseGauss
from model.binaryModel import binaryModel
from model.utils import (initRandomArchitecture, 
                        initGaussianWeights, dataPath,
                        dataPathPerf, my_corrcoef, rescale)

In [ ]:
# The experiment
sim = 'performance_task'
experiment = 'memory_white-noise'

N = 10000
duration = 2000
nb_reservoir = 100 # Number of reservoir par value o sigma
nb_trial = 5 # Number of different state initialization of the same reservoir
K = 16 # Connectivity degree of the graph 
meanWeight = -0.1 # Average weights for the normal distribution
I = 0.2 # Fraction of up neural state (20%) at times t=0

# List of standard deviation sigma (of the normal distribution)
# Predefined sigmas close to the phase transitions
if meanWeight<0:
    sigmas = [0.01  , 0.0133, 0.0167, 0.02, 0.0233, 0.0267, 0.03  ,
            0.0333, 0.0367, 0.04,  0.042, 0.044, 0.046, 0.048, 0.05, 0.052, 0.055, 0.057, 0.06, 0.061, 
            0.0622, 0.0644, 0.065,  0.066, 0.0665, 0.0667, 0.067, 0.0675, 0.068, 0.0685, 
            0.0689, 0.069, 0.0695, 0.07  , 0.0711, 0.0733, 0.0756, 0.0778, 0.08  , 0.085 ,
            0.09  , 0.1   , 0.11  , 0.13  , 0.14  , 0.15  , 0.2   , 0.3   ,
            0.4   , 0.45  , 0.5   , 0.55  , 0.65  , 0.8   , 0.95  , 1.    ,
            2.    ] 
elif meanWeight>0:
    sigmas = [0.08 , 0.1  , 0.12 , 0.15 , 0.16 , 0.2  , 0.24 , 0.28 , 0.3  ,
            0.32 , 0.35 , 0.36 , 0.37 , 0.38 , 0.392, 0.4  , 0.42 , 0.44 ,
            0.45 , 0.48 , 0.5  , 0.52 , 0.56 , 0.6  , 0.62 , 0.64 , 0.68 ,
            0.7  , 0.72 , 0.76 , 0.8  , 0.84 , 0.88 , 0.9  , 0.92 , 0.96 ,
            1.   , 1.2  , 1.5  , 2.   ]

# Values for testing purpose
#nb_sigmas = 2
#exponents = np.linspace(-2, 2, nb_sigmas)
#sigmas = np.power(10, exponents)

print(f'The model will be evaluated on {len(sigmas)} values of sigma(W):')
print('sigma(W) =', sigmas)

In [ ]:
mu = meanWeight
# Parameters of the task --------------------------------------
delays = [-10, -2]#[-18, -14, -10, -6, -2] # Parameter controlling difficulty of the memory task

# Parameter of the training -----------------------------
nbSample = 4000 # Number sample for the white-noise time serie
tTrain = nbSample #  Time of training
nbtrial = 5 # Number of trial for the training and tasks
tDiscard = 500 # Discard 500 time step of transient
tConv = 0 
optionData = {'spikeCount': False, 'indexActive': False} 
kwargs = {'optionRun':optionData} # kwargs for the experiment

# Parameters of the network -----------------------------
inSize = 1 # Number of input neurons
outSize = 1 # Number of output neurons
I = 0.5 # Proportion of neuron of the reservoir to which the input/output is connected
output_option =  'exclusion' # Option to obtain a non-overlapping set of neurons connected to input and readout
kwargsOut = {'option':output_option,
            'I':I}

# Options of the readout --------------------------------
learning = 'online'
epoch = 4000
optimizer = 'pytorch' 
activation = 'sigmoid'
optim = 'adam'
loss = 'MSE'
scheduler = False
alpha = 0.001

if 'ridge' in optimizer:
    kwargs.update({'beta':1E-08})
elif optimizer == 'pseudoinverse':
    kwargs.update({'activation':activation})
kwargs.update({'optionCost':'mse'})
if optimizer == 'ANN':
    kwargs.update({'alpha':1E-04,
                    'nbiter':epoch,
                    'activation':activation})
elif optimizer == 'pytorch':
    kwargs.update({'optionOptim':{'alpha':alpha,
                        'epoch':epoch,
                        'activation':activation,
                        'optim':optim,
                        'loss':loss,
                        'scheduler':scheduler,
                        }
            })

tag = f"{experiment}_{optimizer}_{activation}_output-{output_option}"

# Dictionary of result ---------------------------------
path = dataPathPerf(sim, experiment, optimizer=optimizer, activation=activation, nbtrial=nb_trial)
name = path+f'\dicError_{tag}_nbReservoir{nb_reservoir}_N{N}_K{K}_W{mu}_T{nb_trial}'
try:
    dicResult = np.load(name+'.npy', allow_pickle='TRUE').item()
    corr_delay = dicResult['corr']
    infoReservoir = dicResult['infoReservoir']
    print('File loaded:', name)
except:
    print('File not loaded, will create one:', name)
    corr_delay = {delay: {} for delay in delays}
    infoReservoir = {delay: {} for delay in delays}
    dicResult = {'corr':corr_delay,
                    'delay':delays,
                    'nbtrial':nb_trial,
                    'infoReservoir':infoReservoir,
                    'inputs':'all',
                    'optim': kwargs}

In [ ]:
# Create the model
optionData = {'spikeCount':True, 'indexActive':False} # Option for saving data
brain = binaryModel(N, sim=sim, experiment=experiment)
seedConnectivity = 747
initRandomArchitecture(brain, K, seed=seedConnectivity) # Fixed architecture
print(kwargs)

In [ ]:
## Run the experiment
import glob

for sigma in tqdm(sigmas):
    sigma = np.round(sigma, 5)

    for res_idx in range(nb_reservoir):
        seed=res_idx*100
        print(f'Reservoir #{res_idx}, sigma={sigma}')

        # Initialize weights
        initGaussianWeights(brain, mu, sigma, seed)

        # Input neurons
        optionIn = 'global'
        kwargsIn = {}

        # Input layer initialization
        brain.addInputLayer(inSize=inSize, tag=tag)

        # Readout initialization
        brain.addReadoutLayer(outSize)
        pathBrain = dataPathPerf(sim, experiment, optimizer=optimizer, 
                                 activation=activation, nbtrial=nb_trial)+f'/sigma{sigma}/seed{seed}'

        # Check network has not been run already
        dataFile = glob.glob(pathBrain+"/metadata_"+tag+f'*N{N}_K{K}*_W{meanWeight}_*.npy')
        if len(dataFile) != 0:
            print('Network already run, next...\n \n')
            continue

        # Select delay for the targets signals
        for i, delay in enumerate(delays):
            
            # Initialize dictionary result 
            if corr_delay.get(delay, None) is None:
                corr_delay.update({delay:{}})
                infoReservoir.update({delay:{}})
            if corr_delay[delay].get(sigma, None) is None:
                corr_delay[delay].update({sigma:[]})
                infoReservoir[delay].update({sigma:[]})

            # Check if this task is already performed
            print(f'delay={delay}, sigma={sigma}')
            nb_reservoir_run = len(infoReservoir[delay][sigma])
            print(nb_reservoir_run, 'already runned circuits')
            if nb_reservoir_run >= nb_reservoir:
                print(f'Networks for delay={delay}, sigma={sigma} already run, next...\n \n')
                continue
            elif (nb_reservoir_run < nb_reservoir) and (res_idx < nb_reservoir_run):
                print(f'Network {res_idx} for delay={delay}, sigma={sigma} already run, next...\n \n')
                continue
            print('-----------------------------')
            
            # Initialize inputs and targets for training
            datas = rescale(whiteNoiseGauss(2*nbSample))
            y_train = datas[tDiscard+delay:nbSample+delay]
            x_train = datas[:tTrain] # Input of the network
            t_train = np.arange(0, tTrain)
            inputs_train = [x_train, t_train]

            corrTrials = [] # List to save performance (evulated by the correlation)
            # Compute all trials
            for k in range(nbtrial):
                # Display information
                print(f'Reservoir #{res_idx}, trial #{k}')
                print('********************')
                brain.displayInfoNetwork()
                print('********************')    
                print('White-noise, delay =', delay)

                # Input layer initialization - multiple init
                brain.setArchitectureIn(None, option=optionIn, I=I, seed=k*100, **kwargsIn)
                brain.setWeightsIn(None, distribution='uniform', seed=k*100)
                
                # Readout initialization - multiple init
                brain.setArchitectureOut(None, distribution='uniform', seed=k*100, **kwargsOut)

                ##################################
                # Training
                ##################################
                print('------------------------')
                # Set input and target trains
                u_train = [inputs_train]
                y_train = [y_train]
                
                # TRAIN !
                brain.train(u_trains = u_train,
                            y_targets = y_train,
                            I = I,
                            duration = tTrain,
                            discardTime = tDiscard,
                            convergingTime = tConv,
                            option = learning,
                            optimizer = optimizer,
                            tag = tag,
                            **kwargs)
                print('Mean weight out :', np.mean(brain.Wout), np.std(brain.Wout))
                
                ##################################
                # Task on test set
                ##################################
                brain.reset()
                brain.activationFunction(activation)
                print('------------------------')
                print('Task : white-noise on test data')
                # test set fitting
                datas = rescale(whiteNoiseGauss(2*nbSample))
                y_test = datas[tDiscard+delay:nbSample+delay]
                u_test = datas[:tTrain]
                tTest = len(u_test)
                t_test  = np.arange(0, tTest)
                inputs = (u_test, t_test)
                y_pred, err = brain.read(inputs, duration=tTest, I=I,
                                discardTime=tDiscard, convergingTime=tConv,
                                y_target=y_test, reset=True, tag=tag, **kwargs)
                corrCoeff = my_corrcoef(y_pred, y_test)
                
                ##################################
                # Performance and storing
                ##################################
                # Error for each tau of MG
                print('Total error on that task :')
                print('Corr_coeff =', corrCoeff)
                corrTrials.append(corrCoeff)
                print('Done')
                print('------------------------')
            print('==================================')
            corr_delay[delay][sigma].append(corrTrials)
            infoReservoir[delay][sigma].append(brain.metadata)
            print(brain.metadata)

        # Save metadata
        tag_ = tag + f'_seed{seed}'
        brain.saveData(path=pathBrain, tag=tag_)
        brain.saveMetadata(path=pathBrain, tag=tag_)
        brain.reset(deep=True)
        
        # Save result in a dictionary]
        dicResult['corr'].update(corr_delay)
        dicResult['infoReservoir'].update(infoReservoir)
        np.save(name, dicResult)
    


In [ ]:
# Format all results in a dataframe
import pandas as pd
from model.utils import save_df

def make_csv_corr_vs_attractor(N, mu, K, nbtrial, nbReservoir, 
                               delays, output, optimizer, 
                               activation, sim='performance_task',
                               experiment='prediction_mackey-glass'):
    
    # Save all results
    result_dir = dataPathPerf(sim, experiment)
    commonFolder = os.path.join(result_dir, 'allResults/')
    if not os.path.isdir(commonFolder):
        os.makedirs(commonFolder)
    df_path = commonFolder + f'{experiment}_allReservoirs_N{N}_K{K}_W{mu}_T{nbtrial}'

    path = dataPathPerf(sim, experiment, optimizer=optimizer,
                        activation=activation, nbtrial=nbtrial)
    
    tag = f"{experiment}_{optimizer}_{activation}_output-{output}"
    name = f'/dicError_{tag}_nbReservoir{nbReservoir}_N{N}_K{K}_W{mu}_T{nbtrial}.npy'

    print(path+name)
    dicResult = np.load(path+name, allow_pickle='TRUE').item()
        
    infoReservoirs = dicResult['infoReservoir']
    corrReservoirs = dicResult['corr']
    for delay in delays:
        corrReservoir_sigma = corrReservoirs[delay]
        infoReservoir_sigma = infoReservoirs[delay]
 
        sigmas = []
        seeds = []
        corr_reservoirs = []
        for n, (sigma, corr_list) in enumerate(corrReservoir_sigma.items()):
            infoReservoir_list = infoReservoir_sigma[sigma]
        
            for i, corr in enumerate(corr_list[0:nbReservoir]):
                
                # Load data
                infoReservoir = infoReservoir_list[i]
                sigma = infoReservoir['stdWeights']
                # Update results
                sigmas.append(sigma)
                seeds.append(seed)
                corr = np.array(corr)
                corr = corr[~np.isnan(corr)]
                corr_reservoirs.append(np.mean(corr))
                
        # Create dictionary
        dic_reservoir = {'stdWeights':sigmas,
                       'seed':seeds,
                       f'<C>, delay={delay}':corr_reservoirs,
                       }
        # Save csv
        df = save_df(df_path, dic_reservoir)

    return df

df = make_csv_corr_vs_attractor(N, mu, K, nb_trial, nb_reservoir, 
                               delays, output_option, optimizer, 
                               activation, sim=sim,
                               experiment=experiment)

In [ ]:
df.head()